# Lab 9-4: MNIST NN Deep

In [1]:
#import
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'

#for reproducibility
random.seed(777)
torch.manual_seed(777)
if device =='cuda':
  torch.cuda.manual_seed_all(777)

In [2]:
#parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

#MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                         train=True,
                         transform= transforms.ToTensor(),
                         download= True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform= transforms.ToTensor(),
                         download= True)

#dataset loader
data_loader = torch.utils.data.DataLoader(dataset= mnist_train,
                                          batch_size= batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [3]:
#nn layers
linear1 = torch.nn.Linear(784, 512, bias= True)
linear2 = torch.nn.Linear(512, 512, bias= True)
linear3 = torch.nn.Linear(512, 512, bias= True)
linear4 = torch.nn.Linear(512, 512, bias= True)
linear5 = torch.nn.Linear(512, 10, bias= True)
relu = torch.nn.ReLU()

#xavier initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

#model
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3, relu, linear4, relu, linear5).to(device)

#cost & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device) #Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr= learning_rate)

In [4]:
#train
total_batch = len(data_loader)
for epoch in range(training_epochs):
  avg_cost = 0

  for X, Y in data_loader:
    X = X.view(-1, 28*28).to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
  print('Epoch: {:2d} Cost= {:.9f}'.format(epoch + 1, avg_cost))
print('Learning finished')

Epoch:  1 Cost= 0.212934807
Epoch:  2 Cost= 0.089778394
Epoch:  3 Cost= 0.063994065
Epoch:  4 Cost= 0.049578585
Epoch:  5 Cost= 0.041436791
Epoch:  6 Cost= 0.035340693
Epoch:  7 Cost= 0.029959230
Epoch:  8 Cost= 0.025496801
Epoch:  9 Cost= 0.025775390
Epoch: 10 Cost= 0.020105790
Epoch: 11 Cost= 0.019533116
Epoch: 12 Cost= 0.017164635
Epoch: 13 Cost= 0.019644149
Epoch: 14 Cost= 0.016021365
Epoch: 15 Cost= 0.014124667
Learning finished


In [5]:
#test
with torch.no_grad():
  X_test = mnist_test.data.view(-1, 28*28).float().to(device)
  Y_test = mnist_test.targets.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

  #Get one and predict
  r = random.randint(0, len(mnist_test) - 1)
  X_single_data = mnist_test.data[r:r+1].view(-1, 28*28).float().to(device)
  Y_single_data = mnist_test.targets[r:r+1].to(device)

  print('Label:', Y_single_data.item())
  single_prediction = model(X_single_data)
  print('Prediction:', torch.argmax(single_prediction, 1).item())

Accuracy: 0.9725000262260437
Label: 8
Prediction: 8
